# Required Questions: Please answer completely all five required questions.

## Question 1

* Programmatically download and load into your favorite analytical tool the trip data for September 2015.

* Report how many rows and columns of data you have loaded.

## Question 2

* Plot a histogram of the number of the trip distance (“Trip Distance”).

* Report any structure you find and any hypotheses you have about that structure.

## Question 3

* Report mean and median trip distance grouped by hour of day.

* We’d like to get a rough sense of identifying trips that originate or terminate at one of the NYC area airports. Can you provide a count of how many transactions fit this criteria, the average fare, and any other interesting characteristics of these trips.

## Question 4

* Build a derived variable for tip as a percentage of the total fare.

* Build a predictive model for tip as a percentage of the total fare. Use as much of the data as you like (or all of it). Provide an estimate of performance using an appropriate sample, and show your work.

## Question 5

##### Choose only one of these options to answer for Question 5. There is no preference as to which one you choose. Please select the question that you feel your particular skills and/or expertise are best suited to. If you answer more than one, only the first will be scored.

### Option A: Distributions

* Build a derived variable representing the average speed over the course of a trip.

* Can you perform a test to determine if the average trip speeds are materially the same in all weeks of September? If you decide they are not the same, can you form a hypothesis regarding why they differ?

* Can you build up a hypothesis of average trip speed as a function of time of day?

### Option B: Visualization

* Can you build a visualization (interactive or static) of the trip data that helps us understand intra- vs. inter-borough traffic? What story does it tell about how New Yorkers use their green taxis?

### Option C: Search

*  We’re thinking about promoting ride sharing. Build a function that given point a point P, find the k trip origination points nearest P.

     * For this question, point P would be a taxi ride starting location picked by us at a given LAT-LONG.

     * As an extra layer of complexity, consider the time for pickups, so this could eventually be used for real time ride sharing matching.

     * Please explain not only how this can be computed, but how efficient your approach is (time and space complexity)

### Option D: Anomaly Detection

* What anomalies can you find in the data? Did taxi traffic or behavior deviate from the norm on a particular day/time or in a particular location?

* Using time-series analysis, clustering, or some other method, please develop a process/methodology to identify out of the norm behavior and attempt to explain why those anomalies occurred.

### Option E: Your own curiosity!

* If the data leaps out and screams some question of you that we haven’t asked, ask it and answer it! Use this as an opportunity to highlight your special skills and philosophies.


URL for NYC Taxi Data = 'http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml'


# Libraries Used:

- os
- requests
- urllib

- bs4 (aka BeautifulSoup) for web scraping
- pandas for data manipulation
- IPython (for Markdown)
- Numpy for numerical and statistical operations
- Scipy for numerical and statistical operations
- Bokeh for data visualizaiton
- Matplotlib (but just for its colormaps)
- Sklearn for Random Forest Classifier
- pygam for GAM Regression *This was the only library not installable via Anaconda
- pyproj for map coordinate projection

# ---------------------------------------------------------------------------------------------------------------
# Question 1

* Programmatically download and load into your favorite analytical tool the trip data for September 2015.

* Report how many rows and columns of data you have loaded.

-
Approach: 
1. Use standard **_request_** commands to get the html from the page. 
2. Use **_BeautifulSoup_** to find "href" tags on the page. I used the tutorial at http://www.pythonforbeginners.com/python-on-the-web/web-scraping-with-beautifulsoup as a reference for this. 
3. Filter the list based on green trips and the date to find the URL for the data
4. Use **_urllib_** to download the csv from the s3 bucket. A good reference for this is https://stackoverflow.com/questions/7243750/download-file-from-web-in-python-3 because the python documentation is difficult to use sometimes
5. Use **_Pandas_** to open the file into a dataframe
6. Report the answer nicely in **_Markdown_**. The reference for that is here:https://stackoverflow.com/questions/18878083/can-i-use-variables-on-an-ipython-notebook-markup-cell


In [ ]:
################################################################################
####################### USER DEFINED VARIABLES AND NAMES #######################

## Define the target web site
taxiURL = 'https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

## the values in filterValues can be edited to modify the analysis later if desired
filterValues = ['green_tripdata', '2015-09']

## Define the local name of the CSV file
csvName = 'cabdata.csv'

In [41]:
from os.path import isfile
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from IPython.display import display, Markdown

## Check to see if the csv is already there. If it is then there is no need
## to repeatedly hit their websites to download a large file. 
if not isfile(csvName):
    ################################################################################
    ################## PROGRAMMATICALLY FIND THE URL FOR THE DATA ##################

    ## Send the GET command using Python's requests module
    htmlResponse = requests.get(taxiURL)

    ## Convert the html object to text
    textResponse = htmlResponse.text


    ## Create a Beautiful soup object
    soupResponse = BeautifulSoup(textResponse, 'lxml')

    ## Use a list comprehension with built-in Beautiful Soup methods 
    ## to create a list of all the links
    linkList = [link.get('href') for link in soupResponse.find_all('a')]

    ## Use a list comprehension to filter using filterValues defined above
    ## so that the only entry is the URL for the CSV we want
    csvURL = [item for item in linkList if filterValues[0] in item and filterValues[1] in item][0]

    ################################################################################
    ################## PROGRAMMATICALLY GET THE DATA FROM THE URL ##################

    ## Download the file from csvURL, name it using csvName defined above
    ## and save it in the local working directory
    urllib.request.urlretrieve(csvURL, csvName);

## End if condition
    
################################################################################
###### LOAD THE CSV INTO A DATAFRAME, CHECK THE SIZE, AND PRINT THE ANSWER #####

## Load the csv file into a pandas dataframe
greenDF = pd.read_csv(csvName)

## Determine the shape of the dataframe 
dfRows, dfCols = greenDF.shape

## Print in markdown because it is prettier than simple print commands
mdText = "Answer to Question 1: \n The CSV has **_ {} _** rows and **_ {} _** columns".format(dfRows, dfCols)
display(Markdown(mdText))

Answer to Question 1: 
 The CSV has **_ 1494926 _** rows and **_ 21 _** columns

# ---------------------------------------------------------------------------------------------------------------
# Question 2

* Plot a histogram of the number of the trip distance (“Trip Distance”).

* Report any structure you find and any hypotheses you have about that structure.


-Approach
1. Examine the summary statistics to get a handle on the contents of the data set.
2. Use **_Numpy_** and **_Bokeh_** to compute and plot the desired histogram. 
    - There are easier ways (such as pandas.Dataframe.hist) but I like a bit more control over the visualization. 
    - Although I have read good things, I have not used Bokeh before, so I am taking the opportunity to learn a new package at the same time. 
    - Bokeh has interactive controls, can do a lot of the visual stuff D3 does, and it is REALLY easy at its most basic. 
    - Here is a good Bokeh Histogram reference: https://bokeh.pydata.org/en/latest/docs/gallery/histogram.html. 
    - I kept making histograms, so I defined it as a subroutine which on one hand cleans up my code, but on the other does not allow for overlays and customization. 
2. Zoom into areas of interest in the histogram to examine the structure
    - Examine the sawtooth structure
    - Examine trip distances that equal 0 with regard to fare and trip duration to ascertain whether those values are erroneous
    - Fit a statistical model and provide observations regarding the fit


In [42]:
################################################################################
###################### HISTOGRAM VISUALIZATION SUBROUTINE ######################

def bokehhistogram(values, binsVal, tLabel, xLabel, yLabel):
    from numpy import histogram
    import bokeh
    from bokeh.io import output_notebook
    from bokeh.plotting import figure, show
    from bokeh.layouts import gridplot
    output_notebook()

    0
    ################################################################################
    ############################ GENERATE HISTOGRAM DATA ###########################
    
    ## Use Numpy's histogram to generate the histogram data
    ## numpy.histogram default options: range=None, normed=False, weights=None, density=None¶
    histVals, binEdges = histogram(values, bins= binsVal) 
    
    
    ################################################################################
    ######################## PLOT THE HISTOGRAM USING BOKEH ########################

    ## Initialize the plot
    histPlot = figure(title=tLabel, plot_height = 500, plot_width = 900)
    histPlot.xaxis.axis_label = xLabel
    histPlot.yaxis.axis_label = yLabel

    ## Use the .quad glyphs to represent the bars in the chart
    histPlot.quad(top=histVals, bottom=0, left=binEdges[:-1], right=binEdges[1:],
            fill_color="#036564", line_color="#033649")

    ## Show the plot
    show(histPlot)


In [43]:
def bokehscatter(x,y):
    from numpy import histogram
    import bokeh
    from bokeh.io import output_notebook
    from bokeh.plotting import figure, show
    from bokeh.layouts import gridplot
    output_notebook()
    
    p = figure()
    p.scatter(x[:25000],y[:25000])
    show(p)

In [44]:
################################################################################
############## EXAMINE THE COORDINATES TO DETERMINE MISSING AREAS ##############

bokehscatter(greenDF['Pickup_longitude'], greenDF['Pickup_latitude'])

Loading BokehJS ...

In [45]:
################################################################################
################# EXAMININE SUMMARY STATISTICS ON THE DATAFRAME ################


# ## Print the name of the column and the type
# [print(i, type(greenDF[i].iloc[0])) for i in greenDF]; 

## Split into three dataframes to get descriptive statistics
numDF = greenDF[['Pickup_longitude','Pickup_latitude',
                 'Dropoff_longitude', 'Dropoff_latitude', 
                 'Passenger_count', 'Trip_distance', 'Fare_amount', 
                 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 
                 'Ehail_fee', 'improvement_surcharge', 'Total_amount'] ] 
display(numDF.describe())

## Note: I could convert thses to strings, but it is easier to deal with numerical 
## values in pandas, especially when it comes to cleaning an filtering.
strDF = greenDF[['VendorID','Store_and_fwd_flag', 'RateCodeID', 
                 'Payment_type', 'Trip_type ']] 
display(strDF.describe())

## Convert dates to Pandas timestamp objects
dayDF = greenDF[['lpep_pickup_datetime', 'Lpep_dropoff_datetime']]
dayDF = dayDF.apply(pd.to_datetime)
for col in dayDF:
    greenDF[col] = dayDF[col]
display(dayDF.describe())


,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount
count,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,1.494926e+06,0.0,1.494926e+06,1.494926e+06
mean,-7.383084e+01,4.069114e+01,-7.383728e+01,4.069291e+01,1.370598e+00,2.968141e+00,1.254320e+01,3.512800e-01,4.866408e-01,1.235727e+00,1.231047e-01,NaN,2.920991e-01,1.503215e+01
std,2.776082e+00,1.530882e+00,2.677911e+00,1.476698e+00,1.039426e+00,3.076621e+00,1.008278e+01,3.663096e-01,8.504473e-02,2.431476e+00,8.910137e-01,NaN,5.074009e-02,1.155316e+01
min,-8.331908e+01,0.000000e+00,-8.342784e+01,0.000000e+00,0.000000e+00,0.000000e+00,-4.750000e+02,-1.000000e+00,-5.000000e-01,-5.000000e+01,-1.529000e+01,NaN,-3.000000e-01,-4.750000e+02
25%,-7.395961e+01,4.069895e+01,-7.396782e+01,4.069878e+01,1.000000e+00,1.100000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,NaN,3.000000e-01,8.160000e+00
50%,-7.394536e+01,4.074674e+01,-7.394504e+01,4.074728e+01,1.000000e+00,1.980000e+00,9.500000e+00,5.000000e-01,5.000000e-01,0.000000e+00,0.000000e+00,NaN,3.000000e-01,1.176000e+01
75%,-7.391748e+01,4.080255e+01,-7.391013e+01,4.079015e+01,1.000000e+00,3.740000e+00,1.550000e+01,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,NaN,3.000000e-01,1.830000e+01
max,0.000000e+00,4.317726e+01,0.000000e+00,4.279934e+01,9.000000e+00,6.031000e+02,5.805000e+02,1.200000e+01,5.000000e-01,3.000000e+02,9.575000e+01,NaN,3.000000e-01,5.813000e+02


,VendorID,RateCodeID,Payment_type,Trip_type
count,1.494926e+06,1.494926e+06,1.494926e+06,1.494922e+06
mean,1.782045e+00,1.097653e+00,1.540559e+00,1.022353e+00
std,4.128570e-01,6.359437e-01,5.232935e-01,1.478288e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
50%,2.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00
75%,2.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00
max,2.000000e+00,9.900000e+01,5.000000e+00,2.000000e+00


,lpep_pickup_datetime,Lpep_dropoff_datetime
count,1494926,1494926
unique,1079075,1077210
top,2015-09-20 02:00:32,2015-09-28 00:00:00
freq,9,172
first,2015-09-01 00:00:00,2015-09-01 00:00:00
last,2015-09-30 23:59:58,2015-10-01 23:56:10


In [47]:
################################################################################
############################ GENERATE HISTOGRAM PLOT ###########################

# Pass the trip distance series to the histogram subroutine previously defined
bokehhistogram(greenDF['Trip_distance'], 'auto', 'Histogram of trip distance', 'Miles', 'Frequency')

##For those unfamiliar with Bokeh, remind them that they can interact with the histogram
display(Markdown("Remember that you can interact with this histogram using the tools on the right"))

Loading BokehJS ...

Remember that you can interact with this histogram using the tools on the right

### Observations:
1. This distribution looks like a log normal. You can see it better if you ignore the long end and zoom in between 0 and 50.  
2. If you zoom in on the beginning lots of values that in the bin between 0 and 0.05
3. This distribution has a LOOOONG tail.
4. There is a weird sawtooth oscillation of fractional part of trip distance



In [48]:
################################################################################
################## EXAMINE THE SAWTOOTH STRUCTURE IN THE DATA ##################

divisor = 1

## If you compute the fractional part of each tripDistance value, you can plot a histogram of them
rems = [each % divisor for each in greenDF['Trip_distance']]


bokehhistogram(rems, 100, 'histogram of fractional trip distance', 'miles', 'frequency')

Loading BokehJS ...

### Observations:

1. It looks like there are likely two different trip distance recording methods:
    - one that rounds at the tenth of the mile 
    - one that rounds at the 1/100th of the mile. 

2. Even miles are slightly more frequent than fractional miles
    - Could this be a weird rounding algorithm?
    - According to https://www.nytimes.com/2006/09/17/nyregion/thecity/17fyi.html certain combinations of city blocks correspond to whole miles, so it may just be that the city is laid out such that certain trips result in whole miles. The average NY block is 264 by 900 feet according to wikipedia. 

In [49]:
################################################################################
############################### EXAMINE THE ZEROS ##############################

## The next few plots examine various other columns where the trip distance
## is equal to 0. 0s seem like an error to me, and I wanted to get some
## sense of why they occur.

## Let's filter the dataframe to distance values that equal 0
vsTrips = greenDF[greenDF['Trip_distance'] == 0 ]

## Let's look at a histogram of the fares that were recorded when the distance recorded is 0.
bokehhistogram(vsTrips['Fare_amount'], 'auto', 'histogram of fare when distance = 0' , 'fare ($) ', 'frequency')

Loading BokehJS ...

In [50]:
# Let's compute trip duration and plot a histogram of it when the distance recorded is 0
duTime = [row[2] - row[1] for index, row in vsTrips.iterrows()]
duTimeH = [each.total_seconds()/3600 for each in duTime]
# 
    
## Let's look at a histogram of the fares that were recorded when the distance recorded is 0.
bokehhistogram(duTimeH, 'auto', 'histogram of trip duration when distance = 0' , 'duration (hours) ', 'frequency')

Loading BokehJS ...

In [51]:
## Let's look at the payment codes: 
    # 1= Credit card
    # 2= Cash
    # 3= No charge
    # 4= Dispute
    # 5= Unknown
    # 6= Voided trip

## Histogram of payment type when trip duration was 0
bokehhistogram([int(each) for each in greenDF['Payment_type']], 'auto', 'histogram of Payment Code ID when distance = 0' , 'Payment Code ID', 'frequency')


Loading BokehJS ...

### Observations: 
1. Examining a histogram of the fares when the trip distance was equal to 0 shows that the fares are all over the place. 
    - I did not expect such a wide range of values
    - Some trip distance values may be erroneous
    - Judging simply by the fact that there are negative values, some fares may be erroneous as well. 

2.  Examining a historgram of trip duration when trip distance was equal to 0 shows a distribution of durations that would make sense otherwise if the distance was not 0
    - Trip duration values make physical sense (i.e. no negatives)
    - The distribution of the data is predominately near 0, which makes sense
    - I don't know enough about the rules of cabs (i.e. can they charge a fare for waiting if noone actually travels) to explain these variables with regard to each other. 
    
3. Examining a histogram of payment code when trip distance was equal to 0 seemed reasonable
    -I did not see as many disputes, unknowns, and voids as I would expect given other dubious values.


In [57]:
################################################################################
############################ EXAMINE LOG NORMAL FIT ############################

## I replicated the histogram plotting here, because I wanted to overlay two plots

from scipy import stats 
import numpy as np
from numpy import histogram
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
output_notebook()

## Trim the ends of the distribution of tripDistance
subDF = greenDF[greenDF['Trip_distance']>0]
subDF = subDF[subDF['Trip_distance']<25]


## Use Numpy's histogram to generate the histogram data
## numpy.histogram default options: range=None, normed=False, weights=None, density=None
## NOTE: in this case I am norming because I am comparing the histogram to a predicted pdf curve
## and I want them on the same scale
histVals, binEdges = histogram(subDF['Trip_distance'], bins= 'auto') 
histVals = histVals/histVals.sum()


################################################################################
######################## PLOT THE HISTOGRAM USING BOKEH ########################

## Initialize the plot
histPlot = figure(title= 'Histogram and PDF of truncated trip distance', plot_height = 500, plot_width = 900)
# histPlot.xaxis.axis_label = xLabel
# histPlot.yaxis.axis_label = yLabel

## Use the .quad glyphs to represent the bars in the chart
histPlot.quad(top=histVals, bottom=0, left=binEdges[:-1], right=binEdges[1:],
        fill_color="#036564", line_color="#033649")


################################################################################
########################## PREDICT THE FIT PARAMETERS ##########################


# ## Fit a lognormal curve to the data
shape, loc, scale = stats.lognorm.fit(subDF['Trip_distance'], floc=0) 
mu = np.log(scale) 
sigma = shape 
M = np.exp(mu) 
s = np.exp(sigma) 

## Compute a log Normal curve based on the computed shape, etc.
## For simplicity generate a sequence and predict off that. 
lnX = np.linspace(0, 25, num=400)
lnY = stats.lognorm.pdf(lnX, shape, loc=loc, scale=scale)
lnY = lnY/lnY.sum()

################################################################################
########################### PLOT THE LOGNORMAL CURVE ###########################
histPlot.line(lnX,lnY, line_width=2, color="#B3DE69")

histPlot.xaxis.axis_label = 'Trip Distance (mi)'
histPlot.yaxis.axis_label = 'Normalized frequency'

## Show the plot
show(histPlot)

Loading BokehJS ...

### Observations:

1. As I hypothesized, this data fits a lognormal distribution fairly well. 
    - I cut off the zero values since they are possibly erroneous 
    - I could improve fit by rounding trip distance to 1/10th of a mile, thus removing the sawtooth structure
    - I could tune the model parameters, to improve the fit
    - I could compute an RMS Error of prediction, but for the sake of commenting on the structure, I believe it is sufficient to say that a log normal model generally describes the data. 

In [58]:
################################################################################
############################## CLEAN THE DATAFRAME #############################
## Note: there are probably more "pythonic" ways to do some of the stuff I am '
## doing here, but the time to research and implement is longer than the compute time 

## 1. Keep trips gt 0 and lt 50 to limit edge effects in modeling
cleanDF = greenDF[greenDF['Trip_distance']> 0.1]
cleanDF = cleanDF[cleanDF['Trip_distance']<50]

## 2. Keep total amount >0
cleanDF = cleanDF[cleanDF['Total_amount']>0]

## 3. Keep payment ID 1,2
cleanDF = cleanDF[[int(each)<3 for each in cleanDF['Payment_type']]]

## 4. Round trip distance to 1/10 of a mile
cleanDF['Trip_distance'] = [round(each, 1) for each in cleanDF['Trip_distance']]

# 5. Remove ehail fee since it causes future problems
## I found myself tweaking the values for the cleanup, and getting errors because I had 
## already dropped the row, so I put in error handling
try:
    cleanDF = cleanDF.drop(['Ehail_fee'], axis = 1)
except:
    pass

## Rename the cleaned dataset back to the original name since later sections
## were already written to use this variable
greenDF = cleanDF
greenDF.describe()

,VendorID,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,improvement_surcharge,Total_amount,Payment_type,Trip_type
count,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06,1.454952e+06
mean,1.786688e+00,1.063190e+00,-7.388546e+01,4.072072e+01,-7.389187e+01,4.072237e+01,1.373701e+00,3.024534e+00,1.258681e+01,3.560302e-01,4.924451e-01,1.243540e+00,1.249694e-01,2.954988e-01,1.509945e+01,1.525276e+00,1.014465e+00
std,4.096465e-01,4.893569e-01,1.923577e+00,1.061394e+00,1.779897e+00,9.821901e-01,1.044359e+00,2.992445e+00,9.334968e+00,3.651373e-01,6.099484e-02,2.280728e+00,8.829325e-01,3.647046e-02,1.084457e+01,4.993609e-01,1.193979e-01
min,1.000000e+00,1.000000e+00,-8.331908e+01,0.000000e+00,-8.342784e+01,0.000000e+00,0.000000e+00,1.000000e-01,0.000000e+00,-7.200000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-02,1.000000e+00,1.000000e+00
25%,2.000000e+00,1.000000e+00,-7.395987e+01,4.069913e+01,-7.396833e+01,4.069895e+01,1.000000e+00,1.100000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.300000e+00,1.000000e+00,1.000000e+00
50%,2.000000e+00,1.000000e+00,-7.394573e+01,4.074664e+01,-7.394545e+01,4.074705e+01,1.000000e+00,2.000000e+00,9.500000e+00,5.000000e-01,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.180000e+01,2.000000e+00,1.000000e+00
75%,2.000000e+00,1.000000e+00,-7.391773e+01,4.080216e+01,-7.391072e+01,4.078903e+01,1.000000e+00,3.800000e+00,1.550000e+01,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,3.000000e-01,1.830000e+01,2.000000e+00,1.000000e+00
max,2.000000e+00,6.000000e+00,0.000000e+00,4.317726e+01,0.000000e+00,4.243293e+01,9.000000e+00,4.960000e+01,5.805000e+02,1.000000e+00,5.000000e-01,3.000000e+02,7.500000e+01,3.000000e-01,5.813000e+02,2.000000e+00,2.000000e+00


In [63]:
## Use Numpy's histogram to generate the histogram data
## numpy.histogram default options: range=None, normed=False, weights=None, density=None
## NOTE: in this case I am norming because I am comparing the histogram to a predicted pdf curve
## and I want them on the same scale
histVals, binEdges = histogram(greenDF['Trip_distance'], bins= 'auto') 
histVals = histVals/histVals.sum()

################################################################################
######################## PLOT THE HISTOGRAM USING BOKEH ########################

## Initialize the plot
histPlot = figure(title= 'Histogram and PDF of truncated trip distance', plot_height = 500, plot_width = 900)
# histPlot.xaxis.axis_label = xLabel
# histPlot.yaxis.axis_label = yLabel

## Use the .quad glyphs to represent the bars in the chart
histPlot.quad(top=histVals, bottom=0, left=binEdges[:-1], right=binEdges[1:],
        fill_color="#036564", line_color="#033649")


################################################################################
########################## PREDICT THE FIT PARAMETERS ##########################


# ## Fit a lognormal curve to the data
shape, loc, scale = stats.lognorm.fit(subDF['Trip_distance'], floc=0) 
mu = np.log(scale) 
sigma = shape 
M = np.exp(mu) 
s = np.exp(sigma) 

## Compute a log Normal curve based on the computed shape, etc.
## For simplicity generate a sequence and predict off that. 
lnX = np.linspace(0, 25, num=400)
lnY = stats.lognorm.pdf(lnX, shape, loc=loc, scale=scale)
lnY = lnY/lnY.max()*histVals.max()
################################################################################
########################### PLOT THE LOGNORMAL CURVE ###########################
histPlot.line(lnX,lnY, line_width=2, color="#B3DE69")

histPlot.xaxis.axis_label = 'Trip Distance (mi)'
histPlot.yaxis.axis_label = 'Normalized frequency'

## Show the plot
show(histPlot)

### Observation:
So after cleaning and rounding the distribution did not fit quite as well, but general shape still holds. 

# ---------------------------------------------------------------------------------------------------------------
# Question 3

* Report mean and median trip distance grouped by hour of day.

* We’d like to get a rough sense of identifying trips that originate or terminate at one of the NYC area airports. Can you provide a count of how many transactions fit this criteria, the average fare, and any other interesting characteristics of these trips.

## Approach
##### Question 1
1. Create a derived value for hour of the day using pandas methods
2. Loop through by hour and create a dictionary of the summary statistics by hour. I am using a dictionary because dataframe conversion is particulary easy. Although the dictionary's main key was hour, I added a subset key/value pair for hour because of how the dicitonary would be converted into the dataframe, and if I wanted to use the hour data directly or alter it, it is convenient to have it be its own column and not just the index. 
3. Convert the dictionary to a dataframe, including the orient = 'index' so that it is a long skinny dataframe. I like my columns to be homogenous.
4. Display the dataframe and plot it as a simple line plot
##### 
##### Question 2
1. Using information from the website http://www.nyc.gov/html/exit-page.html?url=https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv, I determined that JFK Airport is zone 132 and LaGuardia Airport is zone 138.... unfortunately the september 2015 data has lat/lon not zones... But this does have a rate code ID where JFK code #2. 


JFK = 40.6413° N, 73.7781° W
LaGuardia = 40.7769° N, 73.8740° W


If I had more time, I could filter on the lat/lon, but I think that is outside the scope of this challenge...

In [64]:
################################################################################
########################### GET HOUR OUT OF TIMESTAMP ##########################

## use Pandas to extract the hour of the day in 24hour notation from the timestamp
greenDF['puHour'] = pd.DatetimeIndex(greenDF['lpep_pickup_datetime']).hour

## We can view a histogram of the new data to visually check the conversion
bokehhistogram(greenDF['puHour'], 'auto', 'Histogram of Pickup Time ' , 'Hour of the day (24hr format) ', 'Frequency ')


Loading BokehJS ...

In [65]:
hours = range(0, 24)

## Use a dictionary to make things easier in terms of conversion to a dataframe
summaryDict = {}
for hour in hours:
    subDF = greenDF[greenDF['puHour']==hour]
    summaryDict[hour] = {'Hour': hour, 'Mean Distance': subDF['Trip_distance'].mean(), 'Median Distance': subDF['Trip_distance'].median()}

## Turn the dictionary into a dataframe to display the data
summaryDF = pd.DataFrame.from_dict(summaryDict, orient = 'index')
display(summaryDF)

,Hour,Mean Distance,Median Distance
0,0,3.165116,2.20
1,1,3.069424,2.20
2,2,3.108351,2.20
3,3,3.262914,2.30
4,4,3.616773,2.40
5,5,4.255468,3.00
6,6,4.174590,3.00
7,7,3.357656,2.20
8,8,3.105979,2.00
9,9,3.062879,2.00


In [67]:
################################################################################
############################ PLOT SUMMARY STATISTICS ###########################

import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
output_notebook()

sumPlot = figure(title = 'Hour of pickup vs. mean and median trip distance')

sumPlot.line(summaryDF['Hour'], summaryDF['Mean Distance'], color = 'red', legend_label = 'Mean Distance')
sumPlot.line(summaryDF['Hour'], summaryDF['Median Distance'], color = 'blue', legend_label = 'Median Distance')

sumPlot.xaxis.axis_label = 'Hour of Day (24 hr notation)'
sumPlot.yaxis.axis_label = 'Trip Distance (mi)'
show(sumPlot)

Loading BokehJS ...

In [68]:
################################################################################
############################ SUBSET TO AIRPORT ZONES ###########################

airPU = greenDF[greenDF['RateCodeID'] == 2]

message = '### Observations:'
display(Markdown(message))


message = 'In my cleaned up data set, the total number of airport trips in Sept 2015 was **_' + str(len(airPU)) + '_** based on the Rate Code ID for the airport'
display(Markdown(message))

message = 'In my cleaned up data set, the mean fare of airport trips in Sept 2015 was **_$' + str(round(airPU['Fare_amount'].mean() ,2)) + '_** based on the Rate Code ID for the airport'
display(Markdown(message))

### Observations:

In my cleaned up data set, the total number of airport trips in Sept 2015 was **_2726_** based on the Rate Code ID for the airport

In my cleaned up data set, the mean fare of airport trips in Sept 2015 was **_$52.0_** based on the Rate Code ID for the airport

In [69]:
################################################################################
################# EXAMINE TRIP DISTANCE FOR AIRPORT RATE CODES #################

bokehhistogram(airPU['Trip_distance'], 10000, 'Histogram of Trip Distance ' , 'Distance', 'Frequency ')

Loading BokehJS ...

### Observations:

I found the multi-modal distribution here interesting. 

Since this is NYC data, the distribution around 18 was not surprising, since that is about the distance from NYC to JFK. 

What was unexpected was the smaller distribution around 0. With more time I would examine the cause of that smaller distribution. 

Additionally, it looks like there might be an additional mode around 21. It would be interesting to examine whether these could be resolved using other criteria such as pick-up/drop off locations, etc. 

# ---------------------------------------------------------------------------------------------------------------
# Question 4

* Build a derived variable for tip as a percentage of the total fare.

* Build a predictive model for tip as a percentage of the total fare. Use as much of the data as you like (or all of it). Provide an estimate of performance using an appropriate sample, and show your work.

## Approach
##### Question 1
1. Since there is no "total fare" variable, I am going to derive a variable with all of the non-tip charges and call it total_fare. Why? Well, if you think about tipping there are three-ish systems that guide how people tip: Flat percentage, Set Amount, and Keep the change, and sometimes they are used together (I want to tip about 20% but will round that to the nearest dollar). In each case we can describe the tip as some function of total_fare (tip = f(total_fare)). total_amount = tip + total fare. So computing a percent tip of the total_amount becomes a less straightforward function, where you will get lots of strange structure. It is easier to see the underlying causes if we compute tip relative to pre-tip total. 
2. I will make scatterplots to examine the tips as they relate to the total_fare computed previously. This validates the behavirors. 

##### Question 2
1. I started with Random Forest classifier, where the class is the tip rounded to the nearest dollar. I did this because I wanted something I had used before and that was a baseline ML technique. 
2. I wanted to try Generalized Additive Models, since I had never worked with them before, and had heard they had good performance in situations where ensemble modeling is desired. Given the different tip types, this seemed to be a good time to try them out. I used this website as a reference https://codeburst.io/pygam-getting-started-with-generalized-additive-models-in-python-457df5b4705f and https://github.com/dswah/pyGAM/blob/master/README.md 
    - I elected to use pyGAM, which is not distributed via Anaconda, and had to pip install it
    - The sampling is random every time, but I am using the same labels as with the random forest.


In [70]:
################################################################################
################### GENERATE DERIVED VARIABLES AND PLOT THEM ###################

import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
output_notebook()


## compute total charges and tip percent
greenDF['totalCharges']= greenDF['Fare_amount']+ greenDF['Extra'] + greenDF['MTA_tax'] + greenDF['Tolls_amount'] + greenDF['improvement_surcharge']
greenDF['tipPct'] = 100* (greenDF['Tip_amount']/greenDF['totalCharges'])

## Note: the entire data set blows out the plotting, so
## I limit it to the first 10,000 points. Good enough to
## visually inspect the data. 
tipScatter = figure()
tipScatter.scatter(greenDF['totalCharges'][:10000], greenDF['Tip_amount'][:10000])
show(tipScatter)

display(Markdown('### Observations:'))
display(Markdown('You can really see the tip structure if you zoom in 0-60 on the x and 0-10 on the y. The straight lines are set amount tipping and slopes are percentage tipping'))

Loading BokehJS ...

### Observations:

You can really see the tip structure if you zoom in 0-60 on the x and 0-10 on the y. The straight lines are set amount tipping and slopes are percentage tipping

In [71]:
## We can plot also plot tip pct of total charges
## Note: the entire data set blows out the plotting, so
## I limit it to the first 10,000 points. Good enough to
## visually inspect the data. 

scatterPlot = figure()
scatterPlot.scatter(greenDF['totalCharges'][:10000], greenDF['tipPct'][:10000])
show(scatterPlot)

message = '### Observations:'
display(Markdown(message))
message = 'If you zoom in, You can see the straight lines at 20, 25, and 30 percent (I guess only waiters get 15?), as well as the curved lines which represent fixed amounts as a percentage of the total fares. What is interesting here is that there are curves below the fixed percentage line. I believe that these represent where people rounded down in their tip'
display(Markdown(message))

### Observations:

If you zoom in, You can see the straight lines at 20, 25, and 30 percent (I guess only waiters get 15?), as well as the curved lines which represent fixed amounts as a percentage of the total fares. What is interesting here is that there are curves below the fixed percentage line. I believe that these represent where people rounded down in their tip

In [72]:
################################################################################
########################### RANDOM FOREST CLASSIFIER ###########################

## But enough delaying... let's use some machine learning to try to predict these values!
## Let's start with something simple like random forest predictors
## Let's create categories for tips. Start with whole number amounts. 

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Subset to what I believe are the truly informative features
X = greenDF[['RateCodeID',
             'Pickup_longitude',
             'Pickup_latitude',
             'Dropoff_longitude',
             'Dropoff_latitude',
             'Passenger_count',
             'Trip_distance',
             'Fare_amount',
             'Extra',
             'MTA_tax',
             'Tip_amount',
             'Tolls_amount',
             'improvement_surcharge',
             'Total_amount',
             'Payment_type',
             'Trip_type ',
             'puHour', 
             'totalCharges']]

## Zeros in the total charges cause math problems, so I am going to omit these from the mode.
X = X[X['totalCharges']>0]

## Since random forest is a classifier, we can categorize tips in whole dollar bins 
totalCharges =[each for each in X['totalCharges']]

tip = [each for each in X['Tip_amount']]

## Something was amiss here, so I broke up the steps. Normally I woul put them all in one step though
int1 = [each/totalCharges[index] for index, each in enumerate(tip)]
int2 = [100* each for each in int1]
int3 = [round(each,0) for each in int2]
y = [str(each) for each in int3]


## Remember to drop the tip
X = X.drop('Tip_amount', axis = 1)

## Choose the number of estimators for the Random forest classifier. If you want to see 
## how other numbers of classifiers do, add them to the array
estArray = [75]

## Iterate through a series of model complexities to determine the break-even point
for estNum in estArray:
    ## Generate test and training data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, train_size= 0.01)

    ## Random forest as explained in SKLearn's documentation'
    ## Initialize a random forest classifier object with 100 estimators
    forestClassifier = RandomForestClassifier(n_estimators=estNum, verbose = 1)

    ## Fit the classifier using the training data
    forestClassifier.fit(X_train, y_train)

    ## Estimate the tips using the test data
    predValues = forestClassifier.predict(X_test)

    ## Ascertain how many matches were correct and display the answer
    correctSum = sum([1 for index, each in enumerate (y_test) if each == predValues[index]])
    pctCorrect = round(100*correctSum/len(y_test),2)
    message = 'This iteration of the model provided **_' + str(pctCorrect)+ '_** accuracy using *' + str(estNum) + '* estimators.'
    display(Markdown(message)) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  1.0min finished


This iteration of the model provided **_80.98_** accuracy using *75* estimators.

In [73]:
## Let's plot those results:
from numpy.random import randn

## Covert the strings back to numbers for ease of plotting
## Since there were quanitized and hard to see, I added a little jitter to 
## make the clumps easier to discern
y_testNum = [float(each)+(randn()/25) for each in y_test[:10000]]
predNum = [float(each) +(randn()/25) for each in predValues[:10000]]

## Make the scatterplot
predFig = figure(title = 'True Tip Pct vs. Predicted Tip Pct')
predFig.scatter(y_testNum, predNum, size = 1.5)
predFig.xaxis.axis_label = 'True rounded tip ($) (with jitter added for clarity)'
predFig.yaxis.axis_label = 'Predicted rounded tip ($) (with jitter added for clarity)'
show(predFig)

### Observations
I am decent results: 80% depending on the number of estimators, the estimator splitting, and the sampling. This is probably pretty good based on the variability of the rides and the lack of amazing indicators. 

In [76]:
################################################################################
########################## GENERALIZED ADDITIVE MODEL ##########################

from pygam import LinearGAM

## Generate test and training sets
X_train, X_test, y_train, y_test = train_test_split( X, int2, test_size=0.99,  random_state=42)

# ## Train the model
# gam = LinearGAM(n_splines=5).gridsearch(X_train, y_train)

# ## Print summary stats on the model
# gam.summary()

# ## Predict the test values
# predValues = gam.predict(X_test)

In [78]:
## Ascertain the error in the predictions
# MPE = 100* (sum([(abs(each-predValues[index]))/each for index, each in enumerate(y_test) if each >0])/len(y_test))

# message = 'Mean Prediction Error = **_' + str(round(MPE,2))+ '_**.'
# display(Markdown(message)) 


# ## Let's plot those results:
# from numpy.random import randn

# ## Covert the strings back to numbers for ease of plotting
# ## Since there were quanitized and hard to see, I added a little jitter to 
# ## make the clumps easier to discern
# y_testNum = [float(each)+(randn()/25) for each in y_test[:10000]]
# predNum = [float(each) +(randn()/25) for each in predValues[:10000]]

# ## Make the scatterplot
# predFig = figure(title = 'True Tip Pct vs. Predicted Tip Pct')
# predFig.scatter(y_testNum, predNum, size = 1.5)
# predFig.xaxis.axis_label = 'True rounded tip ($) (with jitter added for clarity)'
# predFig.yaxis.axis_label = 'Predicted rounded tip ($) (with jitter added for clarity)'
# show(predFig)

### Observations:
I get different results with this. Since I ran this one as a regression, the values are less quantitized. Because of the difference in methods, the error statistics are a little different. 

The results I am getting are of similar quality to the Random Forest, but distributed differently

# ---------------------------------------------------------------------------------------------------------------
# Question 5
Choose only one of these options to answer for Question 5. There is no preference as to which one you choose. Please select the question that you feel your particular skills and/or expertise are best suited to. If you answer more than one, only the first will be scored.


##### Option B: Visualization
Can you build a visualization (interactive or static) of the trip data that helps us understand intra- vs. inter-borough traffic? What story does it tell about how New Yorkers use their green taxis?

## Approach
1. Use Bokeh to do the mapping
    - Being in the GEOINT profession, we have always had GIS software (i.e. ArcGIS) with which to perform cartography, so I have never done any mapping directly via Python or its modules. But I want to see how I *could* do it, so I am trying something new. 
    - I am following the script described in https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10 to create my map. 
2. The map projection was Web mercator, so I needed pyproj to project my lat/lon coordinates into that projection
    - I used https://stackoverflow.com/questions/35315259/using-colormap-with-bokeh-scatter to help with colormapping. The gnuplot colormap is a good one because it had a bit of contrast at the lower values, so I imported MatPlotLib's colormaps with that and coded it usine trip distance

In [79]:
X.describe()

,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tolls_amount,improvement_surcharge,Total_amount,Payment_type,Trip_type,puHour,totalCharges
count,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06,1.454838e+06
mean,1.062882e+00,-7.388546e+01,4.072072e+01,-7.389187e+01,4.072237e+01,1.373710e+00,3.024498e+00,1.258780e+01,3.560580e-01,4.924837e-01,1.249792e-01,2.955220e-01,1.509906e+01,1.525318e+00,1.014388e+00,1.355134e+01,1.385684e+01
std,4.881336e-01,1.923653e+00,1.061435e+00,1.779966e+00,9.822283e-01,1.044388e+00,2.992342e+00,9.334669e+00,3.651380e-01,6.084125e-02,8.829664e-01,3.637795e-02,1.084407e+01,4.993588e-01,1.190834e-01,6.802707e+00,9.621612e+00
min,1.000000e+00,-8.331908e+01,0.000000e+00,-8.342784e+01,0.000000e+00,0.000000e+00,1.000000e-01,0.000000e+00,-7.200000e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-02,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e-02
25%,1.000000e+00,-7.395987e+01,4.069913e+01,-7.396833e+01,4.069895e+01,1.000000e+00,1.100000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,3.000000e-01,8.300000e+00,1.000000e+00,1.000000e+00,9.000000e+00,7.800000e+00
50%,1.000000e+00,-7.394574e+01,4.074664e+01,-7.394545e+01,4.074704e+01,1.000000e+00,2.000000e+00,9.500000e+00,5.000000e-01,5.000000e-01,0.000000e+00,3.000000e-01,1.180000e+01,2.000000e+00,1.000000e+00,1.500000e+01,1.080000e+01
75%,1.000000e+00,-7.391774e+01,4.080216e+01,-7.391073e+01,4.078903e+01,1.000000e+00,3.800000e+00,1.550000e+01,5.000000e-01,5.000000e-01,0.000000e+00,3.000000e-01,1.830000e+01,2.000000e+00,1.000000e+00,1.900000e+01,1.680000e+01
max,6.000000e+00,0.000000e+00,4.317726e+01,0.000000e+00,4.243293e+01,9.000000e+00,4.960000e+01,5.805000e+02,1.000000e+00,5.000000e-01,7.500000e+01,3.000000e-01,5.813000e+02,2.000000e+00,2.000000e+00,2.300000e+01,5.813000e+02


In [80]:
## Use an actual GIS projection library rather than just math... This matters with some measurements

def dd2wm(lon, lat):
    from pyproj import Proj, transform
    lon, lat = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), lon, lat)  # longitude first, latitude second.
    return [lon, lat]

In [81]:
greenDF.head()

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,MTA_tax,Tip_amount,Tolls_amount,improvement_surcharge,Total_amount,Payment_type,Trip_type,puHour,totalCharges,tipPct
2,2,2015-09-01 00:01:50,2015-09-01 00:04:24,N,1,-73.921410,40.766708,-73.914413,40.764687,1,...,0.5,0.50,0.0,0.3,5.80,1,1.0,0,5.3,9.433962
3,2,2015-09-01 00:02:36,2015-09-01 00:06:42,N,1,-73.921387,40.766678,-73.931427,40.771584,1,...,0.5,0.00,0.0,0.3,6.30,2,1.0,0,6.3,0.000000
4,2,2015-09-01 00:00:14,2015-09-01 00:04:20,N,1,-73.955482,40.714046,-73.944412,40.714729,1,...,0.5,0.00,0.0,0.3,6.30,2,1.0,0,6.3,0.000000
5,2,2015-09-01 00:00:39,2015-09-01 00:05:20,N,1,-73.945297,40.808186,-73.937668,40.821198,1,...,0.5,1.36,0.0,0.3,8.16,1,1.0,0,6.8,20.000000
6,2,2015-09-01 00:00:52,2015-09-01 00:05:50,N,1,-73.890877,40.746426,-73.876923,40.756306,1,...,0.5,0.00,0.0,0.3,7.80,1,1.0,0,7.8,0.000000


In [ ]:
# from bokeh.tile_providers import CARTODBPOSITRON_RETINA

# ## Create individual lists with the Web Mercator coordinates
# from matplotlib import cm, colors
# pulonInd = 5
# pulatInd = 6

# circleScale = 1

# test = greenDF.iloc[:10000]
# plArray = [[dd2wm(row[pulonInd], row[pulatInd])[0], dd2wm(row[pulonInd], row[pulatInd])[1]]  for index, row in test.iterrows()]    
# lonArray = [each[0] for each in plArray]
# latArray = [each[1] for each in plArray]
# fareArray =list(test['Trip_distance']**1.25) 
# colors = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*cm.gnuplot(colors.Normalize()(fareArray))]
# sizes = [each for each in fareArray]

# plotHeight = 500
# plotWidth = 900

# xStart = -8250000
# #xEnded = -8200000
# xEnded = xStart + plotWidth*100

# yStart = 4950000
# #yEnded = 5000000
# yEnded = yStart + plotHeight*100

# p = figure(title = "Taxicab pickup locations sized and colored by trip distance", 
#            x_range=(xStart, xEnded), 
#            y_range=(yStart, yEnded),
#            plot_height = plotHeight,
#            plot_width = plotWidth,
#            x_axis_type="mercator", 
#            y_axis_type="mercator")

# p.add_tile(get_provider(Vendors.CARTODBPOSITRON_RETINA))

# p.circle(x = lonArray, 
#         y = latArray, 
#          size = sizes,
#          fill_color=colors, 
#          fill_alpha=0.1,
#          line_color=None)



# output_notebook()
# show(p)

Note: the scroll wheel and pan tool arethe best way to interact with this diagram. using the zoom box distorts the aspect ratio of the map. 


### Observations:
1. I am unsure why the tip of manhattan has no data. The scatterplot at the beginning of Question 2 was intended to examine the data prior to it being cleaned to see if that is the problem, but got the same result. 
    - I could spend a lot of time diagnosing, but I have poured over this so much that I just want to get this in at this point.

2. Trip distance is coded into the map in both color (shorter trips = darker) and size (shorter trips = smaller). 

3. With regard to inter versus intra borough trips: Zooming into Midtown, you can see that there are more inter borough trips, compared to the other boroughs which seemed visually to have a higher diversity of trips.  

Without more data it is difficult to back this up, but this map may also reflect affluence. Given that walking and the Subway are competing modes of transportation, cab rides may be a proxy for some measure of affluence.  



In [ ]:
# def commentbar(stringly):
#     if len(stringly)>70:
#         return stringly
#     else:
        
#         if len(stringly)>0:
#             stringly = " "+stringly+" "
        
#         while len(stringly)<80:
#             stringly = "#"+stringly+"#"

#         if len(stringly)>80:
#             stringly = stringly[:-1]
#         return stringly
    
# print(commentbar(''))    
# print(commentbar('EXAMINE THE COORDINATES TO DETERMINE MISSING AREAS'))


